<a href="https://colab.research.google.com/github/fullpistoladev/relatorio_parcial/blob/main/Pibic_relat%C3%B3rio_parcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download pt_core_news_lg

2023-05-10 17:34:14.615131: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 17:34:15.788856: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

def to_lowercase(data):
    return [line.replace(line, line.lower()) for line in data]

def remove_stopwords(data):
    stop_words = spacy.lang.pt.stop_words.STOP_WORDS
    filtered_tokens = []
    
    for line in data:
        # tokenize the line with spaCy
        doc = nlp(line)
        
        # remove stopwords from the doc
        filtered_tokens_line = [token.text for token in doc if token.text.lower() not in stop_words]
        
        # join the remaining tokens back into a single string
        filtered_line = ' '.join(filtered_tokens_line)
        
        filtered_tokens.append(filtered_line)

    return filtered_tokens

def remove_punct(data):
    #regex para tudo que não for [A-Za-Z0-9] e espaçamento 
    return [re.sub(r'[^\w\s]', ' ', str(line)) for line in data]


def lemmatize(data):
    lemmatizer = WordNetLemmatizer()
    lem_tokens = []
    for line in data:
        tokens = word_tokenize(line, language='portuguese')
        lem_line = []
        for token in tokens:
            lem_token = lemmatizer.lemmatize(token)
            lem_line.append(lem_token)
        lem_string = ' '.join(lem_line)
        lem_tokens.append(lem_string)
    return lem_tokens

def stem(data):
    stemmer = SnowballStemmer('portuguese')
    stem_tokens = []
    for line in data:
        tokens = word_tokenize(line, language='portuguese')
        stem_line = []
        for token in tokens:
            stem_token = stemmer.stem(token)
            stem_line.append(stem_token)
        stem_string = ' '.join(stem_line)
        stem_tokens.append(stem_string)
    return stem_tokens

def preprocess_text(data):
    
    data = remove_punct(data)
    data = to_lowercase(data)
    data = remove_stopwords(data)
    data = lemmatize(data)
    data = stem(data)

    return data

In [ ]:
import spacy
nlp = spacy.load('pt_core_news_lg') 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

path_questions = "drive/MyDrive/Colab Notebooks/data/questions.xlsx"
path_answers = "drive/MyDrive/Colab Notebooks/data/answers.xlsx"

questions = pd.read_excel(path_questions)
answers = pd.read_excel(path_answers)
print(answers)

                          questaoMatrizId                  nomeCandidato  \
0    0b991be1-84ab-40af-8d4b-94d4aa0ac9b2      ANDREON SOUZA DE MEDEIROS   
1    83099bc5-0c76-4633-a6f2-f89d50c673aa      ANDREON SOUZA DE MEDEIROS   
2    e39bcb05-5a01-4b72-81ca-049b438fce69      ANDREON SOUZA DE MEDEIROS   
3    bc490271-4f7e-465a-8e8e-c1d1a5936612      ANDREON SOUZA DE MEDEIROS   
4    0b991be1-84ab-40af-8d4b-94d4aa0ac9b2   ANNY KLARICE FERNANDES SOUZA   
..                                    ...                            ...   
119  bc490271-4f7e-465a-8e8e-c1d1a5936612   WESLLEY BERNARDO DE OLIVEIRA   
120  0b991be1-84ab-40af-8d4b-94d4aa0ac9b2  YURI GUILHERME MOURA DE SOUZA   
121  83099bc5-0c76-4633-a6f2-f89d50c673aa  YURI GUILHERME MOURA DE SOUZA   
122  e39bcb05-5a01-4b72-81ca-049b438fce69  YURI GUILHERME MOURA DE SOUZA   
123  bc490271-4f7e-465a-8e8e-c1d1a5936612  YURI GUILHERME MOURA DE SOUZA   

                              candidatoId  \
0    1cf4ed20-e3bb-43d1-8901-ee3a70aefc05 

In [ ]:
spacy_similarity = pd.DataFrame()


for questionId in range(4):

  temp = answers[answers['questaoMatrizId'] ==  questions['questaoMatrizId'][questionId]]
  temp = temp.dropna(subset=['discursiva.respostaCandidato'])
  expected_answers = nlp(questions['discursiva.expectativaDeResposta'][questionId])
  #filtered_tokens = [token.text for token in expected_answers if not token.is_stop]
  #expected_answers = nlp(' '.join(filtered_tokens))

  for index, row in temp.iterrows():
    #print(row)
    
    student_answer = nlp(row['discursiva.respostaCandidato'])
    #student_answer = [token.text for token in student_answer if not token.is_stop]
    #student_answer_filtered_tokens = nlp(' '.join(student_answer))
    similarity = expected_answers.similarity(student_answer)
    temp.loc[index, 'wordembedding_similarity'] = similarity
  spacy_similarity = pd.concat([spacy_similarity, temp])
    #print(f'{row.nomeCandidato} => {row.questaoMatrizId} => {row.notaQuestao} => {similarity}')


print(spacy_similarity['wordembedding_similarity'])

<ipython-input-7-645283dc7971>:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = expected_answers.similarity(student_answer)
<ipython-input-7-645283dc7971>:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = expected_answers.similarity(student_answer)


0      0.887435
4      0.805268
8      0.277430
12     0.928847
16     0.898586
         ...   
107    0.584143
111    0.661283
115    0.686739
119    0.000000
123    0.719466
Name: wordembedding_similarity, Length: 113, dtype: float64


<ipython-input-7-645283dc7971>:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = expected_answers.similarity(student_answer)


In [ ]:
vectorizer = CountVectorizer()
tf_similarity = pd.DataFrame()

for questionId in range(4):

  temp = answers[answers['questaoMatrizId'] ==  questions['questaoMatrizId'][questionId]]
  temp = temp.dropna(subset=['discursiva.respostaCandidato'])
  expectativa_de_resposta = [questions['discursiva.expectativaDeResposta'][questionId]]
  expectativa_de_resposta = preprocess_text(expectativa_de_resposta)
  tf_matrix = vectorizer.fit_transform(expectativa_de_resposta)
  print(vectorizer.get_feature_names_out())
  
  for index, row in temp.iterrows():
    temp_answer = [temp['discursiva.respostaCandidato'][index]]
    temp_answer = preprocess_text(temp_answer)
    tf_answer = vectorizer.transform(temp_answer)
    count_vectorizer_similarity = cosine_similarity(tf_answer, tf_matrix)
    print(count_vectorizer_similarity)
    temp.loc[index, 'tf_similarity'] = count_vectorizer_similarity[0]
    # print(count_vectorizer_similarity[0])
    #print(f'{row.nomeCandidato} => {row.questaoMatrizId} => {row.notaQuestao} => {count_vectorizer_similarity[0]}')
  tf_similarity = pd.concat([tf_similarity, temp])
print(tf_similarity['tf_similarity'],'\n\n', tf_similarity['notaQuestao'])


['10' '20' 'args' 'argument' 'automovel' 'característ' 'cham' 'checklist'
 'class' 'compil' 'dad' 'depend' 'deriv' 'diferent' 'dinâm' 'estát' 'exat'
 'execut' 'extends' 'há' 'instânc' 'int' 'linguagens' 'main' 'mant'
 'métod' 'new' 'objet' 'ocorr' 'oficin' 'op' 'op1' 'op2' 'oper' 'operat'
 'operating' 'operation' 'orient' 'out' 'parâmetr' 'pass' 'permit'
 'polimorf' 'possibilit' 'princip' 'println' 'process' 'program' 'public'
 'quot' 'ser' 'sobrescrit' 'static' 'string' 'system' 'tip' 'two' 'veicul'
 'void']
[[0.60810142]]
[[0.60093503]]
[[0.29042347]]
[[0.62678801]]
[[0.64379571]]
[[0.63108559]]
[[0.67924863]]
[[0.53096841]]
[[0.34931068]]
[[0.65601227]]
[[0.67340316]]
[[0.24040121]]
[[0.54785657]]
[[0.6767351]]
[[0.23253408]]
[[0.67376219]]
[[0.27794816]]
[[0.65845728]]
[[0.59707503]]
[[0.44910847]]
[[0.31902304]]
[[0.51664355]]
[[0.62547526]]
[[0.27907279]]
[[0.54620612]]
[[0.49193661]]
['cham' 'class' 'cri' 'criaçã' 'defin' 'definiçã' 'descrit' 'funçã'
 'instânc' 'linguag' 'materi

In [ ]:
!pip install xlsxwriter
import xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 12.4 MB/s eta 0:00:00


In [ ]:
writer = pd.ExcelWriter('drive/MyDrive/Colab Notebooks/data/similarities_sheet.xlsx', engine='xlsxwriter')

# write the dataframes to different sheets of the same workbook
tfidf_similarity.to_excel(writer, sheet_name='tfidf_similarity')
tf_similarity.to_excel(writer, sheet_name='tf_similarity')
spacy_similarity.to_excel(writer, sheet_name='spacy_similarity')

# save and close the Excel writer object
writer.save()

In [ ]:
similarities_df = merged_df = pd.merge(pd.merge(tfidf_similarity, tf_similarity, on=['questaoMatrizId', 'nomeCandidato', 'candidatoId', 'discursiva.respostaCandidato', 'notaQuestao'], how='outer'), spacy_similarity, on=['questaoMatrizId', 'nomeCandidato', 'candidatoId', 'discursiva.respostaCandidato', 'notaQuestao'], how='outer')
similarities_df.to_excel('drive/MyDrive/Colab Notebooks/data/similarities.xlsx', index=False)
print(similarities_df)

                          questaoMatrizId                   nomeCandidato  \
0    0b991be1-84ab-40af-8d4b-94d4aa0ac9b2       ANDREON SOUZA DE MEDEIROS   
1    0b991be1-84ab-40af-8d4b-94d4aa0ac9b2    ANNY KLARICE FERNANDES SOUZA   
2    0b991be1-84ab-40af-8d4b-94d4aa0ac9b2       BRUNO DA FONSECA DUMARESQ   
3    0b991be1-84ab-40af-8d4b-94d4aa0ac9b2   DANYLLO ROBEMAR VILELA SANTOS   
4    0b991be1-84ab-40af-8d4b-94d4aa0ac9b2  DANIEL OTAVIANO BEZERRA PONTES   
..                                    ...                             ...   
108  bc490271-4f7e-465a-8e8e-c1d1a5936612        VITORIA MARIA LUCIO DEON   
109  bc490271-4f7e-465a-8e8e-c1d1a5936612          WALCKER DA SILVA GOMES   
110  bc490271-4f7e-465a-8e8e-c1d1a5936612    WASHINGTON WAGNER DAVID CRUZ   
111  bc490271-4f7e-465a-8e8e-c1d1a5936612    WESLLEY BERNARDO DE OLIVEIRA   
112  bc490271-4f7e-465a-8e8e-c1d1a5936612   YURI GUILHERME MOURA DE SOUZA   

                              candidatoId  \
0    1cf4ed20-e3bb-43d1-8901-e